# Lösung der Laplace-Gleichung mit einem Monte-Carlo-Verfahren

Wir lösen
$$ \Delta u = 0 $$
auf dem Einheitsquadrat. Wir konstruieren eine analytische Lösung als Realteil einer holomorphen Funktion:
$$ u(x,y) = \text{Re} f(z) = \text{Re}(\frac{1}{z-2i} - \frac{1}{z+2i}) = \frac{x}{x^2 + (2-y)^2} - \frac{x}{x^2 + (2+y)^2}$$
und lösen das entsprechende Randwertproblem auf einem Gitter mit einem Monte-Carlo-Verfahren.

In [7]:
import numpy as np
from numpy.random import randn, rand
import matplotlib.pyplot as plt
from numba import njit

In [47]:
def u_exact(x, y):
    return x/(x**2 + (y-2)**2) - x/(x**2 + (y+2)**2)

x_0, y_0 = 0.7, 0.8
u_exact(x_0, y_0)

0.2786606870727566

In [50]:
u_boundary = u_exact

#@njit
def mc_step(x_0, y_0):
    u = 0
    dx = 0.05
    eps = 1.e-5
    x, y = x_0, y_0
    particle_inside = True
    while particle_inside:
        rand_step = rand()
        if rand_step < 0.25:
            x -= dx
        elif rand_step < 0.5:
            x += dx
        elif rand_step < 0.75:
            y -= dx
        else:
            y += dx
        if x < eps:
            # particle leaves on left boundary
            u = u_boundary(0, y)
            particle_inside = False
        elif abs(x-1) < eps:
            # particle leaves on right boundary
            u = u_boundary(1, y)
            particle_inside = False
        elif y < eps:
            # particle leaves on lower boundary
            u = u_boundary(x, 0)
            particle_inside = False
        elif abs(y-1) < eps:
            # particle leaves on upper boundary
            u = u_boundary(x, 1)
            particle_inside = False
    #print(x,y,u)
    return u

#@njit
def mc_laplace(x_0, y_0, N=100):
    u = 0
    for k in range(N):
        u += mc_step(x_0, y_0)
    return u/N


In [51]:
%time
mc_laplace(x_0, y_0, 10_000)

CPU times: user 14 μs, sys: 1e+03 ns, total: 15 μs
Wall time: 24.3 μs


0.27810004236020136